In [4]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import re
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

#config method from config.py file
from config import config



In [2]:
#Right now each Engine gets disposed after each call.  
#May be worth revisiting to have it create at beginning, clean up at end


def read_df_from_azure(query):
    
    params = config("database.ini")
    engine = create_engine(URL("postgresql", params["user"], params["password"], params["host"], 5432, params["dbname"]))

    # Context manager makes sure the `Connection` is closed safely and implicitly
    with engine.connect() as conn:
        
        df = pd.read_sql_query(query, conn)
        
        # print(conn.in_transaction()) # False
        # do_something_with(conn)
        
        #trans = conn.begin()
        #print(conn.in_transaction()) # True        
        # do_whatever_with(trans)
        
        #print(conn.closed) # False
    #print('Is Connection with-OUT closed?', conn.closed) # True
    engine.dispose()
    return df
    
    

    
    
def write_df_to_azure(df, table_title = "dummy_table"):
   
    params = config("database.ini")
    engine = create_engine(URL("postgresql", params["user"], params["password"], params["host"], 5432, params["dbname"]))

    with engine.connect() as conn:
        
        df.to_sql(table_title, con = engine, if_exists = "replace", method = "multi")

        conn.closed
    engine.dispose()
    
    
def execute_query_on_azure(query):
    params = config("database.ini")
    engine = create_engine(URL("postgresql", params["user"], params["password"], params["host"], 5432, params["dbname"]))

    
    with engine.connect() as connection:
        result = connection.execute(query)

        
    engine.dispose()

    

In [3]:
df = pd.DataFrame([[1, "Greg", 30], [2, "Frank", 21], [3, "Jimbo", 22]], columns=["ID", "Name", "Age"])

print(df.head())

write_df_to_azure(df, "sql_test")

read_df = read_df_from_azure("SELECT * FROM sql_test;")

read_df.head()





   ID   Name  Age
0   1   Greg   30
1   2  Frank   21
2   3  Jimbo   22


,index,ID,Name,Age
0,0,1,Greg,30
1,1,2,Frank,21
2,2,3,Jimbo,22


In [30]:
execute_query_on_azure("DROP TABLE IF EXISTS sql_test;")



In [23]:
fight_data = pd.read_csv("preprocessed_data.csv")
original_columns = fight_data.columns


#clean up the columnn names to alphanumeric chars using regular expression
#\W+ is the stand in for all alphanumeric values
col_names = {}
for col in fight_data.columns:
    col_names[col] = re.sub(r'\W+', '', col)
    

fight_data.rename(columns = col_names, inplace = True)

# Use to list out the column names for verification
# print([col for col in original_columns])
# print([col for col in fight_data.columns])

fight_data.head()

# write_df_to_azure(fight_data, "fight_data")



,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,...,weight_class_WomensStrawweight,B_Stance_OpenStance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_OpenStance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
0,Red,True,5,0.0,4.0,0.0,9.200000,6.000000,0.200000,0.000000,...,0,0,1,0,0,0,0,1,0,0
1,Red,True,5,0.0,3.0,0.0,14.600000,9.100000,11.800000,7.300000,...,0,0,1,0,0,0,0,0,1,0
2,Red,False,3,0.0,3.0,0.0,15.354839,11.322581,6.741935,4.387097,...,0,0,1,0,0,0,0,1,0,0
3,Blue,False,3,0.0,4.0,0.0,17.000000,14.000000,13.750000,11.000000,...,0,0,0,0,0,1,0,1,0,0
4,Blue,False,3,0.0,1.0,0.0,17.000000,14.500000,2.500000,2.000000,...,0,0,0,0,1,0,0,0,1,0


In [31]:
write_df_to_azure(fight_data, "fight_data")


In [34]:
df = read_df_from_azure("SELECT * FROM fight_data;")

df.head()

,index,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,...,weight_class_WomensStrawweight,B_Stance_OpenStance,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_OpenStance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
0,0,Red,True,5,0.0,4.0,0.0,9.200000,6.000000,0.200000,...,0,0,1,0,0,0,0,1,0,0
1,1,Red,True,5,0.0,3.0,0.0,14.600000,9.100000,11.800000,...,0,0,1,0,0,0,0,0,1,0
2,2,Red,False,3,0.0,3.0,0.0,15.354839,11.322581,6.741935,...,0,0,1,0,0,0,0,1,0,0
3,3,Blue,False,3,0.0,4.0,0.0,17.000000,14.000000,13.750000,...,0,0,0,0,0,1,0,1,0,0
4,4,Blue,False,3,0.0,1.0,0.0,17.000000,14.500000,2.500000,...,0,0,0,0,1,0,0,0,1,0


In [39]:
subset_df = read_df_from_azure("SELECT title_bout FROM fight_data;")

subset_df.head()


,title_bout
0,False
1,True


In [ ]:
execute_query_on_azure("")

In [30]:
#This section retains the work using psycopg2

#Psycopg2 Functions:


# A function that takes in a PostgreSQL query and outputs a pandas database 
#IN: sql_query = text string containing the PostgreSQL query, Database = connection defined by psycopg2.conect() method
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table



#call clean_up_connection to pair a close after this function
def get_cursor_on_database():
    params = config("database.ini")

    conn = psycopg2.connect(**params)
    print("Connection established")
    
    return conn, conn.cursor()


#clean up
def clean_up_connection(conn, cursor):
    conn.commit()
    cursor.close()
    conn.close()
    print("Connection cleaned up")
    
    
###Demo code to demonstrate basic psycopg2 cursor/connection object usable with this setup

conn, cursor = get_cursor_on_database()


#Drop previous table of same name if one exists
cursor.execute("DROP TABLE IF EXISTS inventory;")
print("Finished dropping table (if existed)")

#create a table
cursor.execute("CREATE TABLE inventory (id serial PRIMARY KEY, name VARCHAR(50), quantity INTEGER);")
print("Finished creating table")


#insert some data into the table
cursor.execute("INSERT INTO inventory (name, quantity) VALUES (%s, %s);", ("banana", 150))



# cursor.execute("SELECT Winner FROM fight_data")
cursor.execute("SELECT * FROM inventory;")


rows = cursor.fetchall()

clean_up_connection(conn, cursor)


Connection established
Finished dropping table (if existed)
Finished creating table
Connection cleaned up


Connection Established
Rows Output from fetchall() : [(1, 'banana', 150)]
Data Row = 1, banana, 150


In [6]:
df = pd.DataFrame([[1, "Greg", 30], [2, "Jim", 21]], columns=["ID", "Name", "Age"])


In [43]:
conn, cursor = get_cursor_on_database()


cursor.execute("SELECT * FROM sqlalchemytest")
rows = cursor.fetchall()
for row in rows:
    print("Data Row = {}, {}, {}".format(str(row[0]), str(row[1]), str(row[2])))

clean_up_connection(conn, cursor)

Connection established
Data Row = 0, 1, Greg
Data Row = 1, 2, Jim
Connection cleaned up


In [44]:
engine.dispose()
